## Core

- Prepare files for flux measurements:

    - Primary beam correction
    - Convolution to worse resolution
    - Regrid to common frame
    
- **NOTE** this regridding does not conserve the flux within pixels of same beam, but it conserves the total flux.
- Please use SYNAGE++ instead



In [ ]:
import numpy as np
from astropy.io import ascii, fits
import fluxInt
import pbCorr
import synCl
import convRegr
from prettytable import PrettyTable


fint = fluxInt.flInt()
cvReg = convRegr.cvRegr()
pb = pbCorr.pbcorr()
syn = synCl.synClean()

rootDir = '/home/maccagni/Projects/MFS/FornaxA/contData/Core/'
imDir = rootDir+'inIms/'

fint.rootdir = rootDir
syn.rootdir = rootDir
pb.rootdir = rootDir

aut='KKK'
tableName = rootDir+'coreFileNames'+aut+'.tbl'
tableFileNames= ascii.read(tableName,format='csv')

tableNameOut = rootDir+'coreFileNamesCvRg'+aut+'.tbl'

print tableFileNames.colnames

print '''\t+---------+\n\t Inputs loaded\n\t+---------+'''


In [ ]:
#-------------------------------------------------#
# Primary beam correction
#-------------------------------------------------#

fileNames=np.empty([len(tableFileNames.columns[0])],dtype='object')

for i in xrange(0,len(tableFileNames.columns[0])):
    fileName = rootDir+tableFileNames.columns[0][i]
    if (fileName == '/home/maccagni/Projects/MFS/FornaxA/contData/Core/inIms/ALMA_108GHz.fits' or 
       fileName ==  '/home/maccagni/Projects/MFS/FornaxA/contData/Core/inIms/VLA_1500MHz.fits' or
       fileName== '/home/maccagni/Projects/MFS/FornaxA/contData/Core/inIms/Alma_103GHz.fits' or
       fileName== '/home/maccagni/Projects/MFS/FornaxA/contData/Core/inIms/Alma_113GHz.fits'):
        fileNames[i] = fileName
        continue
    ff=fits.open(fileName)
    dats=ff[0].data
    heads=ff[0].header
    dats=np.squeeze(dats)
    
    if i <2:
        telescope= 'MeerKAT'
    elif i ==2 and i==3:
        telescope= 'VLA'
    elif i ==4: 
        telescope = 'ACA'
        
    fileNamePbCorr = pb.BeamCorrect(fileName,dats,heads,telescope)
    fileNames[i] = fileNamePbCorr
    
print fileNames    
print '''\t+---------+\n\t Primary beam correction done\n\t+---------+'''
 

In [ ]:
#-------------------------------------------------#
# Convolve to worse resolution and reproject
#-------------------------------------------------#
fileBaseRg = '/home/maccagni/Projects/MFS/FornaxA/contData/Core/inIms/VLA_4860MHz_pbcorr.fits'
fileBaseCv = '/home/maccagni/Projects/MFS/FornaxA/contData/Core/inIms/ALMA_108GHz.fits'

a = fits.open(fileBaseCv)
headBeam = a[0].header
mainBeam = np.array([headBeam['BMAJ'],headBeam['BMIN']])
rgFileNames=[]
for i in xrange(0,len(fileNames)):
    
    fileName = fileNames[i]
    #if i == 3:
    #    telescope = 'VLAOPT'
    #else:
    #    telscope= 'good'
    
    outFile = cvReg.convolve(fileName,mainBeam)

    if outFile != fileBaseRg:
        rgFileNames.append(cvReg.regrid(fileBaseRg,outFile))
    else:
        rgFileNames.append(outFile)
    
    #ableFileNames.columns[0][i] = rgFileNames[i]

    #print tableFileNames.columns[0][i]
tableFileNames['FileName'] = tableFileNames['FileName'].astype(str)  
tableFileNames['FileName'] = rgFileNames
#tableFileNames.add_column(rgFileNames, index=0)                                 
#tableFileNames.columns[0] =  rgFileNames

ascii.write(tableFileNames,tableNameOut,format='csv', overwrite=True)

print '''\t+---------+\n\t Convolution done\n\t+---------+'''
